In [1]:
from src.data.data_batcher import get_batch_generator
from src.data.vocab import get_glove
import os
import re

In [2]:
train_context_path = os.path.join("data", "train.context")
train_qn_path = os.path.join("data", "train.question")
train_ans_path = os.path.join("data", "train.span")
dev_context_path = os.path.join("data", "dev.context")
dev_qn_path = os.path.join("data", "dev.question")
dev_ans_path = os.path.join("data", "dev.span")

In [3]:
glove_path = os.path.join("data", "glove.6B.{}d.txt".format(100))

emb_matrix, word2id, id2word = get_glove(glove_path, 100)

Loading GLoVE vectors from file: data/glove.6B.100d.txt


In [4]:
_PAD = '<pad>'
_UNK = '<unk>'
characters = 'abcdefghijklmnopqrstuvwxyz0123456789,.<>:;{}[]\|=+_-()*&?^%$#@!`~\n/\"\'\\'
characters = list([_PAD, _UNK]) + list(characters)
char2id = {character: i for i, character in enumerate(characters)}

In [5]:
klm = get_batch_generator(word2id, char2id, train_context_path, 
                          train_qn_path, train_ans_path, 
                          batch_size=32, context_len=600, 
                          question_len=50, character_len=20, discard_long=False)

In [ ]:
P = next(klm)

In [2]:
import torch as th
import torch
from torch import nn as nn
from torch.nn import functional as F
from src.layers.embedding import CharacterConvEmbedding
from src.layers.attention import BasicAttention, BiAttention, CoAttention
from src.layers.network import EmbedText, Encoder, DynamicPointingDecoder

In [7]:
char_args = {'vocab_size': 71, 'embedding_dim': 8, 'padding_idx':0}
char_conv_args = {'char_embed_dim':8, 'embedding_dim':100, 'kernel_sizes':[2, 3, 4], 'num_filters':50, 'use_cuda':True}
word_args = {'vocab_size':4e5, 'embedding_dim':100, 'embeddings':emb_matrix, 'trainable':False, 'padding_idx':0}

In [8]:
embedtext = EmbedText(word_args, char_args, shared=True)
encoder = Encoder(word_args, char_conv_args, shared=True)
attention = CoAttention(hidden_dim=200, use_cuda=True)
output = DynamicPointingDecoder(400, pool_size=16, p=0.25, max_iter=1, use_cuda=True)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
enc_optim = optim.Adam(filter(lambda p: p.requires_grad, encoder.parameters()),lr=LR)
dec_optim = optim.Adam(decoder.parameters(),lr=LR)

In [9]:
for i, batch in enumerate(klm):
    
    question_words_embed, context_words_embed, question_characters_embed, context_character_embed = embedtext(
        th.from_numpy(batch.qn_ids).type(torch.LongTensor), 
          th.from_numpy(batch.context_ids).type(torch.LongTensor), 
          th.from_numpy(batch.qn_char_ids).type(torch.LongTensor), 
          th.from_numpy(batch.context_char_ids).type(torch.LongTensor))
    
    question_words = question_words_embed.cuda().type(torch.cuda.FloatTensor)
    context_words = context_words_embed.cuda().type(torch.cuda.FloatTensor)
    question_characters = question_characters_embed.cuda().type(torch.cuda.FloatTensor)
    context_characters = context_character_embed.cuda().type(torch.cuda.FloatTensor)
    
    encoder = Encoder(word_args, char_conv_args, shared=True)
    question_encoded, context_encoded = encoder(question_words, context_words, question_characters, context_characters)
    
    attention = CoAttention(hidden_dim=200, use_cuda=True)
    U = attention(question_encoded, context_encoded)
    
    alpha, beta, entropies = output(U, is_training=False)
    
    print (i)

Refilling batches...
Refilling batches took 2.18 seconds
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
Refilling batches...
Refilling batches took 2.23 seconds
160
161
162
163
164
165
166
167
168


KeyboardInterrupt: 